#Acıklama

### Yöntem

# Kütüphaneler

In [33]:


import pandas as pd
import numpy as np

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

In [34]:
# siniflandirma icin model

from tensorflow.keras.models import Sequential
from keras import layers
from keras import models
from tensorflow.keras.layers import Conv2D, Activation,Flatten, Dense,MaxPooling2D, MaxPool2D, Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical 
from keras import regularizers, optimizers
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [35]:


import os
import librosa
import librosa.display
import glob 
import skimage

In [36]:
df = pd.read_csv("../input/urbansound8k/UrbanSound8K.csv")

df.head()


,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [37]:
df.groupby('class').slice_file_name.count()

class
air_conditioner     1000
car_horn             429
children_playing    1000
dog_bark            1000
drilling            1000
engine_idling       1000
gun_shot             374
jackhammer          1000
siren                929
street_music        1000
Name: slice_file_name, dtype: int64

#####  sinif

* classID:
0 = air_conditioner
1 = car_horn
2 = children_playing
3 = dog_bark
4 = drilling
5 = engine_idling
6 = gun_shot
7 = jackhammer
8 = siren
9 = street_music


# Database olusturma

In [38]:
feature = []
label = []

def parser(row):
    
    for i in range(8732):
        file_name = '../input/urbansound8k/fold' + str(df["fold"][i]) + '/' + df["slice_file_name"][i]
        
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        
        mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)        
        feature.append(mels)
        label.append(df["classID"][i])
    return [feature, label]

In [39]:
temp = parser(df)

In [40]:
temp = np.array(temp)
data = temp.transpose()

In [41]:
X_ = data[:, 0]
Y = data[:, 1]
X = np.empty([8732, 128])

In [42]:
for i in range(8732):
    X[i] = (X_[i])

In [43]:
Y = to_categorical(Y)

In [44]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

In [45]:
X_train = X_train.reshape(6549, 16, 8, 1)
X_test = X_test.reshape(2183, 16, 8, 1)

In [46]:
input_dim = (16, 8, 1)

#  Keras Model  

In [47]:
model = Sequential()

In [48]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1024, activation = "tanh"))
model.add(Dense(10, activation = "softmax"))

In [49]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [50]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 50, validation_data = (X_test, Y_test))

Train on 6549 samples, validate on 2183 samples
Epoch 1/100
6549/6549 [==============================] - 5s 749us/sample - loss: 1.5372 - accuracy: 0.4755 - val_loss: 1.3642 - val_accuracy: 0.5318
Epoch 2/100
6549/6549 [==============================] - 4s 597us/sample - loss: 1.1548 - accuracy: 0.6088 - val_loss: 1.0900 - val_accuracy: 0.6340
Epoch 3/100
6549/6549 [==============================] - 4s 575us/sample - loss: 0.9750 - accuracy: 0.6761 - val_loss: 1.0111 - val_accuracy: 0.6793
Epoch 4/100
6549/6549 [==============================] - 4s 576us/sample - loss: 0.8496 - accuracy: 0.7212 - val_loss: 0.9327 - val_accuracy: 0.6990
Epoch 5/100
6549/6549 [==============================] - 4s 581us/sample - loss: 0.7459 - accuracy: 0.7525 - val_loss: 0.8980 - val_accuracy: 0.7073
Epoch 6/100
6549/6549 [==============================] - 4s 586us/sample - loss: 0.6935 - accuracy: 0.7761 - val_loss: 0.8614 - val_accuracy: 0.7279
Epoch 7/100
6549/6549 [==============================] - 4

In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 2, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)             

In [52]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print(score)

2183/2183 [==============================] - 0s 192us/sample - loss: 0.8109 - accuracy: 0.8708
[0.8108597579375899, 0.87082]


In [53]:
preds = np.argmax(predictions, axis = 1)

In [54]:
result = pd.DataFrame(preds)
result.to_csv("UrbanSound8kResults.csv")